## Utils

In [ ]:
def pprint(text):
    import textwrap
    wrapped_text = textwrap.fill(text, width=100) 
    print(wrapped_text)

## LLMs

In [36]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

emb_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from llama_index.llms.ollama import Ollama  

chat_model = Ollama(model="qwen2:7b")

## Researcher: An Agent Workflow

Let's create an agent workflow that would: 
 1. take a topic to write a blogpost about and subdivide it into section
 2. In the beginning define the variables and terminology that is going to be used for explaining (mathematical) science concepts  
    a. Briefly explain those concepts if they are complicated? 
 3. For each section: 
    1. Look up websites that explain the concepts mentioned in each section. 
    2. Select websites that best fit to our topic and explain the concepts of this section. 
    3. Download the selected websites and process their data (make a note that you need to refence them)
    4. Decide which website is the best at explaining each concept 
    5. 

In [ ]:
from llama_index.core.agent.workflow import ReActAgent, FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context

import mlflow

mlflow.set_experiment(experiment_name="Test")
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.llama_index.autolog()


def get_relevant_webpages(query: str) -> list:
    """
    Gets the relevant webpages urls for a query.

    Args:
        query (str): what to search online on www.

    Returns:
        list: list of url links (websites) related to the query 
    """
    search_ggg = DDGS()
    results = search_ggg.text(query)
    return results

web_search_tool = FunctionTool.from_defaults(
    fn=get_relevant_webpages,
    name="get_relevant_webpages",
    description="Useful for getting a list of relevant webpages (url links) for a particular query. " \
    "Together with the web links the list also include a short information about the answer for the query."
)

search_agent = ReActAgent(
    name = "Web Searcher", 
    description = "Search the web give links to the relevant pages it found",
    system_prompt = "",
    tools=[web_search_tool],
    verbose=True,
    llm=chat_model,
)

ctx = Context(search_agent)
answer = await search_agent.run("History of Prussia, search online with the web search tool", ctx=ctx)
print(answer)
pprint(answer.response.blocks[0].text)

2025/07/19 12:42:25 INFO mlflow.tracking.fluent: Experiment with name 'Test' does not exist. Creating a new experiment.
/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/inspect.py:592: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  value = getter(object, key)
/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/inspect.py:592: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  value = getter(object, key)
/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/inspect.py:592: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on 

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step


/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/site-packages/mlflow/llama_index/tracer.py:290: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  if params_str := metadata.json(exclude_unset=True):


Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The history of Prussia begins with the Hohenzollerns being made rulers of the Margraviate of Brandenburg in 1518. In 1529, they secured the reversion of the Duchy of Pomerania after a series of conflicts and subsequently acquired its eastern part following the Peace of Westphalia. In 1618, the electors of Brandenburg also inherited the Duchy of Prussia since then.

Prussia became an independent duchy under the Hohenzollern family in the early modern period. Its king, Frederick I, was crowned as King of Prussia in 1701. The country experienced ethnic and political divisions during its history.

These are just some highlights from my search about the rich history of Prussia that spanned several centuries and involved various rulers and territorial changes.
The history of Prussia begins with the Hohenzollerns being made rulers of the Margraviate of
Brandenburg in

Trace(trace_id=72d4c593c8ba4eefb6ae4d808e602bdc)